In [1]:
#初始化配置
import os
import sys
from train.train_helper import get_args_parser
from train.bert_lstm_ner_cg_estimator import train

args = get_args_parser()
# print(args)
os.environ['CUDA_VISIBLE_DEVICES'] = args.device_map
args.task_name="NER"
args.do_train=True
args.do_eval=True
args.do_predict=True
args.data_dir="D:/project/python_project/bert-lstm-crf-ner/data"
args.vocab_file="D:/project/python_project/bert-lstm-crf-ner/bert\chinese_L-12_H-768_A-12/vocab.txt"
args.bert_config_file="D:/project/python_project/bert-lstm-crf-ner/bert/chinese_L-12_H-768_A-12/bert_config.json"
args.init_checkpoint="D:/project/python_project/bert-lstm-crf-ner/bert/chinese_L-12_H-768_A-12/bert_model.ckpt"
args.max_seq_length=128
args.train_batch_size=32
args.learning_rate=2e-5
args.num_train_epochs=3.0
args.output_dir="D:/project/python_project/bert-lstm-crf-ner/output"
#打印相关参数
param_str = '\n'.join(['%20s = %s' % (k, v) for k, v in sorted(vars(args).items())])
print('usage: %s\n%20s   %s\n%s\n%s\n' % (' '.join(sys.argv), 'ARG', 'VALUE', '_' * 50, param_str))

usage: c:\python36\lib\site-packages\ipykernel_launcher.py -f C:\Users\daizhutao\AppData\Roaming\jupyter\runtime\kernel-daaf6e0b-c6df-4bb8-9a1f-6eca1276fdac.json
                 ARG   VALUE
__________________________________________________
          batch_size = 64
    bert_config_file = D:/project/python_project/bert-lstm-crf-ner/bert/chinese_L-12_H-768_A-12/bert_config.json
                cell = lstm
               clean = True
                clip = 0.5
            data_dir = D:/project/python_project/bert-lstm-crf-ner/data
          device_map = 0
             do_eval = True
       do_lower_case = True
          do_predict = True
            do_train = True
        dropout_rate = 0.5
     filter_adam_var = True
     init_checkpoint = D:/project/python_project/bert-lstm-crf-ner/bert/chinese_L-12_H-768_A-12/bert_model.ckpt
          label_list = None
       learning_rate = 2e-05
           lstm_size = 128
      max_seq_length = 128
                 ner = ner
          num_layers =

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
#sys.path.append('/home/idm/dzt/models/nlp_model/bert-lstm-crf-ner/')
import collections
import os
import numpy as np
import tensorflow as tf
import codecs
import pickle

from train import tf_metrics
from bert import modeling
from bert import optimization
from bert import tokenization

# import

from train.models import create_model, InputFeatures, InputExample

__version__ = '0.1.0'

__all__ = ['__version__', 'DataProcessor', 'NerProcessor', 'write_tokens', 'convert_single_example',
           'filed_based_convert_examples_to_features', 'file_based_input_fn_builder',
           'model_fn_builder', 'train']



class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_data(cls, input_file):
        """Reads a BIO data."""
        with codecs.open(input_file, 'r', encoding='utf-8') as f:
            lines = []
            words = []
            labels = []
            for line in f:
                contends = line.strip()
                tokens = contends.split(' ')
                if len(tokens) == 2:
                    words.append(tokens[0])
                    labels.append(tokens[1])
                else:
                    if len(contends) == 0:
                        l = ' '.join([label for label in labels if len(label) > 0])
                        w = ' '.join([word for word in words if len(word) > 0])
                        lines.append([l, w])
                        words = []
                        labels = []
                        continue
                if contends.startswith("-DOCSTART-"):
                    words.append('')
                    continue
            return lines


class NerProcessor(DataProcessor):
    def __init__(self, output_dir):
        self.labels = set()
        self.output_dir = output_dir

    def get_train_examples(self, data_dir):
        return self._create_example(
            self._read_data(os.path.join(data_dir, "train.txt")), "train"
        )

    def get_dev_examples(self, data_dir):
        return self._create_example(
            self._read_data(os.path.join(data_dir, "dev.txt")), "dev"
        )

    def get_test_examples(self, data_dir):
        return self._create_example(
            self._read_data(os.path.join(data_dir, "test.txt")), "test")

    def get_labels(self, labels=None):
        #传入参数可能是labels文件路径，也可能是逗号分隔的labels文本
        if labels is not None:
            try:
                # 支持从文件中读取标签类型
                if os.path.exists(labels) and os.path.isfile(labels):
                    with codecs.open(labels, 'r', encoding='utf-8') as fd:
                        for line in fd:
                            self.labels.append(line.strip())
                else:
                    # 否则通过传入的参数，按照逗号分割
                    self.labels = labels.split(',')
                self.labels = set(self.labels) # to set
            except Exception as e:
                print(e)
        # 通过读取train文件获取标签的方法会出现一定的风险。
        if os.path.exists(os.path.join(self.output_dir, 'label_list.pkl')):
            with codecs.open(os.path.join(self.output_dir, 'label_list.pkl'), 'rb') as rf:
                self.labels = pickle.load(rf)
        else:
            if len(self.labels) > 0:
                #pkl文件不存在，就按照读取的标签集合加上一些其他标签，写入pkl
                self.labels = self.labels.union(set(["X", "[CLS]", "[SEP]"]))
                with codecs.open(os.path.join(self.output_dir, 'label_list.pkl'), 'wb') as rf:
                    pickle.dump(self.labels, rf)
            else:
                #如果什么都没有，都按照代码写好的标签集合
                self.labels = ["O", 'B-TIM', 'I-TIM', "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "X", "[CLS]", "[SEP]"]
        return self.labels

    def _create_example(self, lines, set_type):
        #比如set_type是train，就表示训练数据
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            #使用bert内部的tokenization包，把字符串转化成unicode
            text = tokenization.convert_to_unicode(line[1])
            label = tokenization.convert_to_unicode(line[0])
            # if i == 0:
            #     print('label: ', label)
            #模型训练的输入类，guid为唯一数据id
            examples.append(InputExample(guid=guid, text=text, label=label))
        return examples

    def _read_data(self, input_file):
        """Reads a BIO data."""
        with codecs.open(input_file, 'r', encoding='utf-8') as f:
            lines = []
            words = []
            labels = []
            for line in f:
                contends = line.strip()
                tokens = contends.split(' ')
                if len(tokens) == 2:
                    words.append(tokens[0])
                    labels.append(tokens[-1])
                else:
                    if len(contends) == 0 and len(words) > 0:
                        label = []
                        word = []
                        for l, w in zip(labels, words):
                            if len(l) > 0 and len(w) > 0:
                                label.append(l)
                                self.labels.add(l)
                                word.append(w)
                        lines.append([' '.join(label), ' '.join(word)])
                        words = []
                        labels = []
                        continue
                if contends.startswith("-DOCSTART-"):
                    continue
            return lines


def write_tokens(tokens, output_dir, mode):
    """
    将序列解析结果写入到文件中
    只在mode=test的时候启用
    :param tokens:
    :param mode:
    :return:
    """
    if mode == "test":
        path = os.path.join(output_dir, "token_" + mode + ".txt")
        wf = codecs.open(path, 'a', encoding='utf-8')
        for token in tokens:
            if token != "**NULL**":
                wf.write(token + '\n')
        wf.close()


def convert_single_example(ex_index, example, label_list, max_seq_length, tokenizer, output_dir, mode):
    """
    将一个样本进行分析，然后将字转化为id, 标签转化为id,然后结构化到InputFeatures对象中
    :param ex_index: index
    :param example: 一个样本
    :param label_list: 标签列表
    :param max_seq_length:
    :param tokenizer:
    :param output_dir
    :param mode:
    :return:
    """
    label_map = {}
    # 1表示从1开始对label进行index化
    for (i, label) in enumerate(label_list, 1):
        label_map[label] = i
    # 保存label->index 的map
    if not os.path.exists(os.path.join(output_dir, 'label2id.pkl')):
        with codecs.open(os.path.join(output_dir, 'label2id.pkl'), 'wb') as w:
            pickle.dump(label_map, w)

    #因为从训练数据读取后，字，标签标记，都是用空格分隔的
    textlist = example.text.split(' ')
    labellist = example.label.split(' ')
    tokens = []
    labels = []
    for i, word in enumerate(textlist):
        # 分词，如果是中文，就是分字,但是对于一些不在BERT的vocab.txt中得字符会被进行WordPice处理（例如中文的引号），可以将所有的分字操作替换为list(input)
        token = tokenizer.tokenize(word)
        tokens.extend(token)
        label_1 = labellist[i]
        for m in range(len(token)):
            if m == 0:
                labels.append(label_1)
            else:  # 一般不会出现else，因为只有一个词
                labels.append("X")
    # tokens = tokenizer.tokenize(example.text)
    # 序列截断
    if len(tokens) >= max_seq_length - 1:
        tokens = tokens[0:(max_seq_length - 2)]  # -2 的原因是因为序列需要加一个句首和句尾标志
        labels = labels[0:(max_seq_length - 2)]
    ntokens = []
    segment_ids = []  #segment_ids的作用是？？？
    label_ids = []
    ntokens.append("[CLS]")  # 句子开始设置CLS 标志
    segment_ids.append(0)
    # append("O") or append("[CLS]") not sure!
    label_ids.append(label_map["[CLS]"])  # O OR CLS 没有任何影响，不过我觉得O 会减少标签个数,不过句首和句尾使用不同的标志来标注，使用LCS 也没毛病
    for i, token in enumerate(tokens):
        ntokens.append(token)
        segment_ids.append(0)
        label_ids.append(label_map[labels[i]])
    ntokens.append("[SEP]")  # 句尾添加[SEP] 标志
    segment_ids.append(0)
    # append("O") or append("[SEP]") not sure!
    label_ids.append(label_map["[SEP]"])
    #调用bert内部的token2id函数，把token转化成bert使用的tokenid
    input_ids = tokenizer.convert_tokens_to_ids(ntokens)  # 将序列中的字(ntokens)转化为ID形式
    input_mask = [1] * len(input_ids)   #mask是隐藏的token，用于模型训练
    # label_mask = [1] * len(input_ids)
    # padding, 使用
    #小于序列长度的，进行补全操作
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)
        # we don't concerned about it!
        label_ids.append(0)
        ntokens.append("**NULL**")
        # label_mask.append(0)
    # print(len(input_ids))
    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length
    assert len(label_ids) == max_seq_length
    # assert len(label_mask) == max_seq_length

    # 打印部分样本数据信息
    if ex_index < 5:
        tf.logging.info("*** Example ***")
        tf.logging.info("guid: %s" % (example.guid))
        tf.logging.info("tokens: %s" % " ".join(
            [tokenization.printable_text(x) for x in tokens]))
        tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
        tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
        tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
        tf.logging.info("label_ids: %s" % " ".join([str(x) for x in label_ids]))
        # tf.logging.info("label_mask: %s" % " ".join([str(x) for x in label_mask]))

    # 结构化为一个类，使用自定义类保存训练index化数据，
    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_ids=label_ids,
        # label_mask = label_mask
    )
    # mode='test'的时候才有效，把token写入文件
    write_tokens(ntokens, output_dir, mode)
    return feature


def filed_based_convert_examples_to_features(
        examples, label_list, max_seq_length, tokenizer, output_file, output_dir, mode=None):
    """
    将数据转化为TF_Record 结构，作为模型数据输入
    :param examples:  样本
    :param label_list:标签list
    :param max_seq_length: 预先设定的最大序列长度
    :param tokenizer: tokenizer 对象
    :param output_file: tf.record 输出路径
    :param mode:
    :return:
    """
    writer = tf.python_io.TFRecordWriter(output_file)
    # 遍历训练数据
    for (ex_index, example) in enumerate(examples):
        if ex_index % 5000 == 0:
            tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))
        # 对于每一个训练样本,将一个样本进行分析，然后将字转化为id, 标签转化为id,然后结构化到InputFeatures对象中
        # 返回的是bert模型训练需要的index化token，label，mask，segment等信息
        feature = convert_single_example(ex_index, example, label_list, max_seq_length, tokenizer, output_dir, mode)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()#除了记录k,v，还会记录k放入的顺序
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["label_ids"] = create_int_feature(feature.label_ids)
        # features["label_mask"] = create_int_feature(feature.label_mask)
        # tf.train.Example/Feature 是一种协议，方便序列化？？？
        # Example中存放features特征，放入example是为了便于把特征序列化存储
        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())


def file_based_input_fn_builder(input_file, seq_length, is_training, drop_remainder,batch_size):
    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([seq_length], tf.int64),
        # "label_ids":tf.VarLenFeature(tf.int64),
        # "label_mask": tf.FixedLenFeature([seq_length], tf.int64),
    }

    def _decode_record(record, name_to_features):
        #把recod变成字典example？,可能写入，读取都是按照tensorflow的某个标准
        example = tf.parse_single_example(record, name_to_features)
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t
        return example

    d = tf.data.TFRecordDataset(input_file)
    if is_training:
        d = d.repeat()
        d = d.shuffle(buffer_size=300)
    #通过map函数，调用_decode_record，把int64的数据转化成int32的数据，通过apply，把数据转化成batch的形式
    d = d.apply(tf.data.experimental.map_and_batch(lambda record: _decode_record(record, name_to_features),
                                                   batch_size=batch_size,
                                                   num_parallel_calls=8,  # 并行处理数据的CPU核心数量，不要大于你机器的核心数
                                                   drop_remainder=drop_remainder))
    d = d.prefetch(buffer_size=4)

    return d


def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, args):
    """
    构建模型
    :param bert_config:
    :param num_labels:
    :param init_checkpoint:
    :param learning_rate:
    :param num_train_steps:
    :param num_warmup_steps:
    :param use_tpu:
    :param use_one_hot_embeddings:
    :return:
    """

    def model_fn(features, labels, mode, params):
        tf.logging.info("*** Features ***")
        for name in sorted(features.keys()):
            tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))
        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        print('shape of input_ids', input_ids.shape)
        # label_mask = features["label_mask"]
        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        # 使用参数构建模型,input_idx 就是输入的样本idx表示，label_ids 就是标签的idx表示
        #全部损失，分数，，预测类别
        total_loss, logits, trans, pred_ids = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
            num_labels, False, args.dropout_rate, args.lstm_size, args.cell, args.num_layers)
        # tf.summary.scalar('total_loss', total_loss)
        # tf.summary.scalar('logits',logits)
        # tf.summary.scalar('trans',trans)
        # tf.summary.scalar('pred_ids',pred_ids)
        #所有需要训练的变量
        tvars = tf.trainable_variables()
        # 加载BERT模型，assignmen_map，加载的预训练变量值
        if init_checkpoint:
            (assignment_map, initialized_variable_names) = \
                 modeling.get_assignment_map_from_checkpoint(tvars,
                                                             init_checkpoint)
            tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        # 打印变量名
        # logger.info("**** Trainable Variables ****")
        #
        # # 打印加载模型的参数
        # for var in tvars:
        #     init_string = ""
        #     if var.name in initialized_variable_names:
        #         init_string = ", *INIT_FROM_CKPT*"
        #     logger.info("  name = %s, shape = %s%s", var.name, var.shape,
        #                     init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:
            #train_op = optimizer.optimizer(total_loss, learning_rate, num_train_steps)
            train_op = optimization.create_optimizer(
                 total_loss, learning_rate, num_train_steps, num_warmup_steps, False)
            hook_dict = {}
            hook_dict['loss'] = total_loss
            hook_dict['global_steps'] = tf.train.get_or_create_global_step()
            logging_hook = tf.train.LoggingTensorHook(
                hook_dict, every_n_iter=args.save_summary_steps)

            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                training_hooks=[logging_hook])

        elif mode == tf.estimator.ModeKeys.EVAL:
            # 针对NER ,进行了修改
            def metric_fn(label_ids, pred_ids):
                return {
                    "eval_loss": tf.metrics.mean_squared_error(labels=label_ids, predictions=pred_ids),
                }

            eval_metrics = metric_fn(label_ids, pred_ids)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metric_ops=eval_metrics
            )
        else:
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions=pred_ids
            )
        return output_spec

    return model_fn


# def load_data():
#     processer = NerProcessor()
#     processer.get_labels()
#     example = processer.get_train_examples(FLAGS.data_dir)
#     print()

def get_last_checkpoint(model_path):
    if not os.path.exists(os.path.join(model_path, 'checkpoint')):
        tf.logging.info('checkpoint file not exits:'.format(os.path.join(model_path, 'checkpoint')))
        return None
    last = None
    with codecs.open(os.path.join(model_path, 'checkpoint'), 'r', encoding='utf-8') as fd:
        for line in fd:
            line = line.strip().split(':')
            if len(line) != 2:
                continue
            if line[0] == 'model_checkpoint_path':
                last = line[1][2:-1]
                break
    return last


def adam_filter(model_path):
    """
    去掉模型中的Adam相关参数，这些参数在测试的时候是没有用的
    :param model_path:
    :return:
    """
    last_name = get_last_checkpoint(model_path)
    if last_name is None:
        return
    sess = tf.Session()
    imported_meta = tf.train.import_meta_graph(os.path.join(model_path, last_name + '.meta'))
    imported_meta.restore(sess, os.path.join(model_path, last_name))
    need_vars = []
    for var in tf.global_variables():
        if 'adam_v' not in var.name and 'adam_m' not in var.name:
            need_vars.append(var)
    saver = tf.train.Saver(need_vars)
    saver.save(sess, os.path.join(model_path, 'model.ckpt'))

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = args.device_map

#一个处理的类，包括训练数据的输入等
processors = {
    "ner": NerProcessor
}
#载入bert配置文件
bert_config = modeling.BertConfig.from_json_file(args.bert_config_file)

#检查序列的最大长度是否超出范围
if args.max_seq_length > bert_config.max_position_embeddings:
    raise ValueError(
        "Cannot use sequence length %d because the BERT model "
        "was only trained up to sequence length %d" %
        (args.max_seq_length, bert_config.max_position_embeddings))

# 在re train 的时候，才删除上一轮产出的文件，在predicted 的时候不做clean
if args.clean and args.do_train:
    if os.path.exists(args.output_dir):
        def del_file(path):
            ls = os.listdir(path)
            for i in ls:
                c_path = os.path.join(path, i)
                if os.path.isdir(c_path):
                    del_file(c_path)
                else:
                    os.remove(c_path)
        try:
            del_file(args.output_dir)
        except Exception as e:
            print(e)
            print('pleace remove the files of output dir and data.conf')
            exit(-1)

#check output dir exists
if not os.path.exists(args.output_dir):
    os.mkdir(args.output_dir)

#通过output_dir初始化数据处理类，processor
processor = processors[args.ner](args.output_dir)
#通过bert字典，初始化bert自带分词类
tokenizer = tokenization.FullTokenizer(
    vocab_file=args.vocab_file, do_lower_case=args.do_lower_case)


train_examples = None
eval_examples = None
num_train_steps = None
num_warmup_steps = None

#一般都是True
if args.do_train and args.do_eval:
    # 加载训练数据,train和dev，会自动拼接文件夹和train.txt
    #返回的训练数据是一个list，每个元素是两个字符串，空格分隔字，空格分隔字标记，并写入训练examples类中
    train_examples = processor.get_train_examples(args.data_dir)
    #训练步数
    num_train_steps = int(
        len(train_examples) *1.0 / args.batch_size * args.num_train_epochs)
    if num_train_steps < 1:
        raise AttributeError('training data is so small...')
    #
    num_warmup_steps = int(num_train_steps * args.warmup_proportion)

    tf.logging.info("***** Running training *****")
    tf.logging.info("  Num examples = %d", len(train_examples))
    tf.logging.info("  Batch size = %d", args.batch_size)
    tf.logging.info("  Num steps = %d", num_train_steps)
    #读取验证集
    eval_examples = processor.get_dev_examples(args.data_dir)

    # 打印验证集数据信息
    tf.logging.info("***** Running evaluation *****")
    tf.logging.info("  Num examples = %d", len(eval_examples))
    tf.logging.info("  Batch size = %d", args.batch_size)

#获取标签集合，是一个list

INFO:tensorflow:***** Running training *****
INFO:tensorflow:  Num examples = 20864
INFO:tensorflow:  Batch size = 64
INFO:tensorflow:  Num steps = 978
INFO:tensorflow:***** Running evaluation *****
INFO:tensorflow:  Num examples = 2318
INFO:tensorflow:  Batch size = 64


In [4]:
print(len(train_examples))

20864


In [5]:
label_list = processor.get_labels()
print(label_list)

{'I-PER', 'O', 'B-LOC', 'X', 'I-LOC', 'B-PER', 'B-ORG', '[CLS]', 'I-ORG', '[SEP]'}


In [6]:
with tf.name_scope('input'):
    input_ids = tf.placeholder(tf.int32, [None, args.max_seq_length])
    input_mask = tf.placeholder(tf.int32, [None, args.max_seq_length])
    segment_ids  = tf.placeholder(tf.int32, [None, args.max_seq_length])
    label_ids = tf.placeholder(tf.int32, [None, args.max_seq_length])
#对参数赋值，对于训练模型来说
is_training=True
num_labels=len(label_list) + 1
init_checkpoint = args.init_checkpoint
learning_rate = args.learning_rate
total_loss, logits, trans, pred_ids, accuracy= create_model(
        bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
        num_labels, False, args.dropout_rate, args.lstm_size, args.cell, args.num_layers)
#输出loss的smmary
tf.summary.scalar('total_loss', total_loss)
tf.summary.scalar('accuracy', accuracy)
#加载预训练隐变量
tvars = tf.trainable_variables()
# 加载BERT模型，assignmen_map，加载的预训练变量值
if init_checkpoint:
    (assignment_map, initialized_variable_names) = \
        modeling.get_assignment_map_from_checkpoint(tvars,
                                                    init_checkpoint)
    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
#优化loss
train_op = optimization.create_optimizer(
    total_loss, learning_rate, num_train_steps, num_warmup_steps, False)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.


ValueError: Tried to convert 'size' to a tensor and failed. Error: Shapes must be equal rank, but are 0 and 1
	From merging shape 0 with other shapes. for 'Slice_2/packed' (op: 'Pack') with input shapes: [], [?].

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
train_file = os.path.join(args.output_dir, "train.tf_record")
#ok
if not os.path.exists(train_file):
    filed_based_convert_examples_to_features(
        train_examples, label_list, args.max_seq_length, tokenizer, train_file, args.output_dir)

# 2.读取record 数据，组成batch，把上一部输出到文件的训练数据读取
train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=args.max_seq_length,
    is_training=True,
    drop_remainder=True,
    batch_size=args.batch_size)

In [ ]:
eval_file = os.path.join(args.output_dir, "eval.tf_record")
if not os.path.exists(eval_file):
    filed_based_convert_examples_to_features(
        eval_examples, label_list, args.max_seq_length, tokenizer, eval_file, args.output_dir)
#构建验证集数据
eval_input_fn = file_based_input_fn_builder(
    input_file=eval_file,
    seq_length=args.max_seq_length,
    is_training=False,
    drop_remainder=False,
    batch_size=args.batch_size)
train_input=train_input_fn.make_one_shot_iterator()
sess = tf.InteractiveSession()
max_step=500
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('./log', sess.graph)
meta_train_data = train_input.get_next()
#参数batch_size是64，train_batch_size是32，不知道train_batch_size是什么用的
#------------------解决FailedPreconditionError:问题，初始化所有变量，不知道这样会不会影响初始化的bert预训练变量------------------
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

In [ ]:
    for i in range(max_step):

        #把tensor转化为numpy输入
        train_data=sess.run([meta_train_data])[0]
        sess.run(train_op,feed_dict={input_ids:train_data['input_ids'],input_mask:train_data['input_mask'],
                                     segment_ids:train_data['segment_ids'],label_ids:train_data['label_ids']})
        if i%10==0:
            train_summary, accu = sess.run([merged,accuracy], feed_dict={input_ids:train_data['input_ids'],input_mask:train_data['input_mask'],
                                     segment_ids:train_data['segment_ids'],label_ids:train_data['label_ids']})
            train_writer.add_summary(train_summary, i)
            print('accuracy %s at %s'%(accu,i))
    train_writer.close()